In [4]:
import cv2
from typing import *
import numpy as np
from insightface.app import FaceAnalysis
from face.schemas.Detector import *

class InsightFaceClient(Detector):
    def __init__(self):
        self.model = FaceAnalysis("buffalo_l")
        self.model.prepare(ctx_id=0, det_size=(640, 640))
    
    def detect_faces(self, img) -> List[FacialAreaRegion]:
        faces = self.model.get(img)
        detected_faces = []

        for face in faces:
            x1, y1, x2, y2 = map(int, face.bbox)
            facial_region = FacialAreaRegion(
                x=x1, y=y1, w=x2-x1, h=y2-y1,
                left_eye=tuple(map(int, face.kps[0])),
                right_eye=tuple(map(int, face.kps[1])),
                nose=tuple(map(int, face.kps[2])),
                left_mouth=tuple(map(int, face.kps[3])),
                right_mouth=tuple(map(int, face.kps[4])),
                confidence=float(face.det_score)
            )
            detected_faces.append(facial_region)

        return detected_faces

In [6]:
insightface_client = InsightFaceClient()

Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: C:\Users\dinhd/.insightface\models\buffalo_l\1k3d68.onnx landmark_3d_68 ['None', 3, 192, 192] 0.0 1.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: C:\Users\dinhd/.insightface\models\buffalo_l\2d106det.onnx landmark_2d_106 ['None', 3, 192, 192] 0.0 1.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: C:\Users\dinhd/.insightface\models\buffalo_l\det_10g.onnx detection [1, 3, '?', '?'] 127.5 128.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: C:\Users\dinhd/.insightface\models\buffalo_l\genderage.onnx genderage ['None', 3, 96, 96] 0.0 1.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: C:\Users\dinhd/.insightface\models\buffalo_l\w600k_r50.onnx recognition ['None', 3, 112, 112] 127.

In [9]:
img = cv2.imread("test2.jpg")
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

faces = insightface_client.detect_faces(img)

# Vẽ bounding box và các điểm trên ảnh
for face in faces:
    x1, y1, w, h = face.x, face.y, face.w, face.h
    cv2.rectangle(img, (x1, y1), (x1 + w, y1 + h), (255, 0, 0), 2)

    keypoints = [face.left_eye, face.right_eye, face.nose, face.left_mouth, face.right_mouth]
    colors = [(0, 255, 0), (0, 255, 0), (0, 0, 255), (255, 0, 0), (255, 0, 0)]  # Màu khác nhau

    for point, color in zip(keypoints, colors):
        cv2.circle(img, point, 5, color, -1)

# Hiển thị ảnh
cv2.imshow("Detected Faces", cv2.cvtColor(img, cv2.COLOR_RGB2BGR))
cv2.waitKey(0)
cv2.destroyAllWindows()
